In [1]:
import os
import ipywidgets as widgets
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from phaser.utils import load_labelencoders, bin2bool


hash_dist_dir = r"./demo_outputs/"

# Load the label encoders
le = load_labelencoders(filename="LabelEncoders", path=hash_dist_dir)

# Get values to construct triplets
TRANSFORMS = le["t"].classes_
METRICS = le["m"].classes_
ALGORITHMS = le["a"].classes_

df_d = pd.read_csv(os.path.join(hash_dist_dir , "distances.csv.bz2"))
df_h = pd.read_csv(os.path.join(hash_dist_dir , "hashes.csv.bz2"))

# convert the strings to arrays
for _a in ALGORITHMS:
    df_h[_a] = df_h[_a].apply(bin2bool)


# # Create a label encoder for the class labels
# le_c = LabelEncoder()
# le_c.classes_ = np.array(["Inter (0)", "Intra (1)"])
intra_df = df_d[df_d["class"] == 1]
inter_df = df_d[df_d["class"] == 0]


In [2]:
from phaser.evaluation import dist_stats
from phaser.plotting import  hist_fig, kde_ax
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, VBox
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# macro_tables = []
# for transform in TRANSFORMS[:-1]:
#     stats = macro_stats(
#         data=intra_df, le_a=le_a, le_m=le_m, transform=transform, style=False
#     )
#     macro_tables.append(str(stats))



def plot_image(transform, mode):
    data = df_h
    if transform != 'Select':
        if mode == "inter":
            fig = hist_fig(inter_df, label_encoding=le, transform=transform, interactive=True)
        elif mode == "intra":
            fig = hist_fig(intra_df, label_encoding=le, transform=transform, interactive=True)

#define widgets
tselect = widgets.Dropdown(
        options=TRANSFORMS[:-1],
        description='Transform'
    )
mselect = widgets.Dropdown(
        options=METRICS,
        description='Metric'
    )
aselect = widgets.Dropdown(
        options=ALGORITHMS,
        description='Algorithm'
    )
modeselect = widgets.Dropdown(
        options=["inter", "intra"],
        description='Comparison Mode'
    )

# #interact function in isolation
h = interactive(plot_image, transform=tselect, mode=modeselect)
display(h)

#print(df)

### KDE multi
def kde_plot_multi(transform):
    if transform != 'Select':

        #t_label = le_a.transform(np.array(transform).ravel()
        n_cols = len(METRICS)
        n_rows = len(ALGORITHMS)
        # Subset data
        fig, axes = plt.subplots(ncols=n_cols, nrows=n_rows, figsize=(6,6), constrained_layout=True, 
                                 sharex=True, sharey=False)
                                 
        for col_i, metric in enumerate(METRICS):
            for row_i, algo in enumerate(ALGORITHMS):
                    # Transform strings to labels
                    m_label = le["m"].transform(np.array(metric).ravel())
                    a_label = le["a"].transform(np.array(algo).ravel())

                    # Subset data and get the distances for the chosen transformation
                    _X = df_d.query(f"algo=={a_label} and metric == {m_label}")
                    kde_ax(_X, transform, label_encoding=le, fill=True, title=f"{a_label.capitalize()} - {m_label.capitalize()}"", ax=axes[row_i, col_i]
)

### KDE Single
def kde_plot(metric, algorithm, transform):
    if transform != 'Select':


        m_label = le["m"].transform(np.array(metric).ravel())
        a_label = le["a"].transform(np.array(algorithm).ravel())
        #t_label = le_a.transform(np.array(transform).ravel())


        # Subset data
        print()
        data = df_d.query(f"algo == {a_label} and metric == {m_label}").copy()
        fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(4,3), constrained_layout=True)
        ax = kde_ax(data, transform, label_encoding=le, fill=True, title=f"{algorithm} - {metric} - {transform}", ax=ax
)        

print(ALGORITHMS, METRICS,TRANSFORMS)

#kde_plot(metric="Hamming", algorithm="pHash", transform="Flip_Horizontal")
z = interactive(kde_plot, metric=mselect, algorithm=aselect, transform=tselect)
display(z)

k = interactive(kde_plot_multi, transform=tselect)
display(k)


interactive(children=(Dropdown(description='Transform', options=('Border_bw20_bc255.0.0', 'Border_bw30_bc255.0…

['colour' 'pdq' 'phash'] ['Cosine' 'Hamming'] ['Border_bw20_bc255.0.0' 'Border_bw30_bc255.0.0' 'Flip_Horizontal' 'orig']


interactive(children=(Dropdown(description='Metric', options=('Cosine', 'Hamming'), value='Cosine'), Dropdown(…

interactive(children=(Dropdown(description='Transform', options=('Border_bw20_bc255.0.0', 'Border_bw30_bc255.0…